In [1]:
import zstandard
import os
import json
import sys
import csv
from datetime import datetime
import logging.handlers

# put the path to the input file
input_file = r"./subreddits/Mechwarrior5_submissions.zst"
# put the name or path to the output file. The file extension from below will be added automatically
output_file = r"./subreddits/output"
# the format to output in, pick from the following options
#   zst: same as the input, a zstandard compressed ndjson file. Can be read by the other scripts in the repo
#   txt: an ndjson file, which is a text file with a separate json object on each line. Can be opened by any text editor
#   csv: a comma separated value file. Can be opened by a text editor or excel
# WARNING READ THIS: if you use txt or csv output on a large input file without filtering out most of the rows, the resulting file will be extremely large. Usually about 7 times as large as the compressed input file
output_format = "csv"
# override the above format and output only this field into a text file, one per line. Useful if you want to make a list of authors or ids. See the examples below
# any field that's in the dump is supported, but useful ones are
#   author: the username of the author
#   id: the id of the submission or comment
#   link_id: only for comments, the fullname of the submission the comment is associated with
#   parent_id: only for comments, the fullname of the parent of the comment. Either another comment or the submission if it's top level
single_field = None
# the fields in the file are different depending on whether it has comments or submissions. If we're writing a csv, we need to know which fields to write.
# The filename from the torrent has which type it is, but you'll need to change this if you removed that from the filename
is_submission = "submission" in input_file

# only output items between these two dates
from_date = datetime.strptime("2005-01-01", "%Y-%m-%d")
to_date = datetime.strptime("2025-01-01", "%Y-%m-%d")

# the field to filter on, the values to filter with and whether it should be an exact match
# some examples:
#
# return only objects where the author is u/watchful1 or u/spez
# field = "author"
# values = ["watchful1","spez"]
# exact_match = True
#
# return only objects where the title contains either "stonk" or "moon"
# field = "title"
# values = ["stonk","moon"]
# exact_match = False
#
# return only objects where the body contains either "stonk" or "moon". For submissions the body is in the "selftext" field, for comments it's in the "body" field
# field = "selftext"
# values = ["stonk","moon"]
# exact_match = False
#
#
# filter a submission file and then get a file with all the comments only in those submissions. This is a multi step process
# add your submission filters and set the output file name to something unique
# input_file = "redditdev_submissions.zst"
# output_file = "filtered_submissions"
# output_format = "csv"
# field = "author"
# values = ["watchful1"]
#
# run the script, this will result in a file called "filtered_submissions.csv" that contains only submissions by u/watchful1
# now we'll run the script again with the same input and filters, but set the output to single field. Be sure to change the output file to a new name, but don't change any of the other inputs
# output_file = "submission_ids"
# single_field = "id"
#
# run the script again, this will result in a file called "submission_ids.txt" that has an id on each line
# now we'll remove all the other filters and update the script to input from the comments file, and use the submission ids list we created before. And change the output name again so we don't override anything
# input_file = "redditdev_comments.zst"
# output_file = "filtered_comments"
# single_field = None  # resetting this back so it's not used
# field = "link_id"  # in the comment object, this is the field that contains the submission id
# values_file = "submission_ids.txt"
# exact_match = False  # the link_id field has a prefix on it, so we can't do an exact match
#
# run the script one last time and now you have a file called "filtered_comments.csv" that only has comments from your submissions above
# if you want only top level comments instead of all comments, you can set field to "parent_id" instead of "link_id"

In [2]:
field = "author"
values = ["watchful1","spez"]
# if you have a long list of values, you can put them in a file and put the filename here. If set this overrides the value list above
# if this list is very large, it could greatly slow down the process
values_file = None
exact_match = True


# sets up logging to the console as well as a file
log = logging.getLogger("bot")
log.setLevel(logging.INFO)
log_formatter = logging.Formatter('%(asctime)s - %(levelname)s: %(message)s')
log_str_handler = logging.StreamHandler()
log_str_handler.setFormatter(log_formatter)
log.addHandler(log_str_handler)
if not os.path.exists("logs"):
	os.makedirs("logs")
log_file_handler = logging.handlers.RotatingFileHandler(os.path.join("logs", "bot.log"), maxBytes=1024*1024*16, backupCount=5)
log_file_handler.setFormatter(log_formatter)
log.addHandler(log_file_handler)

In [3]:
def write_line_zst(handle, line):
	handle.write(line.encode('utf-8'))
	handle.write("\n".encode('utf-8'))


def write_line_json(handle, obj):
	handle.write(json.dumps(obj))
	handle.write("\n")


def write_line_single(handle, obj, field):
	if field in obj:
		handle.write(obj[field])
	else:
		log.info(f"{field} not in object {obj['id']}")
	handle.write("\n")


def write_line_csv(writer, obj, is_submission):
	output_list = []
	output_list.append(str(obj['score']))
	output_list.append(datetime.fromtimestamp(obj['created_utc']).strftime("%Y-%m-%d"))
	if is_submission:
		output_list.append(obj['title'])
	output_list.append(f"u/{obj['author']}")
	output_list.append(f"https://www.reddit.com{obj['permalink']}")
	if is_submission:
		if obj['is_self']:
			if 'selftext' in obj:
				output_list.append(obj['selftext'])
			else:
				output_list.append("")
		else:
			output_list.append(obj['url'])
	else:
		output_list.append(obj['body'])
	writer.writerow(output_list)


def read_and_decode(reader, chunk_size, max_window_size, previous_chunk=None, bytes_read=0):
	chunk = reader.read(chunk_size)
	bytes_read += chunk_size
	if previous_chunk is not None:
		chunk = previous_chunk + chunk
	try:
		return chunk.decode()
	except UnicodeDecodeError:
		if bytes_read > max_window_size:
			raise UnicodeError(f"Unable to decode frame after reading {bytes_read:,} bytes")
		log.info(f"Decoding error with {bytes_read:,} bytes, reading another chunk")
		return read_and_decode(reader, chunk_size, max_window_size, chunk, bytes_read)


def read_lines_zst(file_name):
	with open(file_name, 'rb') as file_handle:
		buffer = ''
		reader = zstandard.ZstdDecompressor(max_window_size=2**31).stream_reader(file_handle)
		while True:
			chunk = read_and_decode(reader, 2**27, (2**29) * 2)

			if not chunk:
				break
			lines = (buffer + chunk).split("\n")

			for line in lines[:-1]:
				yield line.strip(), file_handle.tell()

			buffer = lines[-1]

		reader.close()

In [4]:
if single_field is not None:
    log.info("Single field output mode, changing output file format to txt")
    output_format = "txt"
output_path = f"{output_file}.{output_format}"

writer = None
if output_format == "zst":
    log.info("Output format set to zst")
    handle = zstandard.ZstdCompressor().stream_writer(open(output_path, 'wb'))
elif output_format == "txt":
    log.info("Output format set to txt")
    handle = open(output_path, 'w', encoding='UTF-8')
elif output_format == "csv":
    log.info("Output format set to csv")
    handle = open(output_path, 'w', encoding='UTF-8', newline='')
    writer = csv.writer(handle)
else:
    log.error(f"Unsupported output format {output_format}")
    sys.exit()

if values_file is not None:
    values = []
    with open(values_file, 'r') as values_handle:
        for value in values_handle:
            values.append(value.strip().lower())
    log.info(f"Loaded {len(values)} from values file")
else:
    values = [value.lower() for value in values]  # convert to lowercase

file_size = os.stat(input_file).st_size
file_bytes_processed = 0
created = None
matched_lines = 0
bad_lines = 0
total_lines = 0
counter = 0
max_lines_to_process = 100
for line, file_bytes_processed in read_lines_zst(input_file):
    counter += 1
    if counter > max_lines_to_process:
        break
    log.info(f"reading line {str(line)}")
    total_lines += 1
    if total_lines % 1000 == 0:
        log.info(f"{created.strftime('%Y-%m-%d %H:%M:%S')} : {total_lines:,} : {matched_lines:,} : {bad_lines:,} : {file_bytes_processed:,}:{(file_bytes_processed / file_size) * 100:.0f}%")

    try:
        obj = json.loads(line)
        created = datetime.utcfromtimestamp(int(obj['created_utc']))

        if created < from_date:
            continue
        if created > to_date:
            continue

        # field_value = obj[field].lower()
        # matched = False
        # for value in values:
        #     if exact_match:
        #         if value == field_value:
        #             matched = True
        #             break
        #     else:
        #         if value in field_value:
        #             matched = True
        #             break
        # if not matched:
        #     continue

        matched_lines += 1
        if output_format == "zst":
            write_line_zst(handle, line)
        elif output_format == "csv":
            write_line_csv(writer, obj, is_submission)
        elif output_format == "txt":
            if single_field is not None:
                write_line_single(handle, obj, single_field)
            else:
                write_line_json(handle, obj)
    except (KeyError, json.JSONDecodeError) as err:
        bad_lines += 1
handle.close()

2023-08-04 09:37:07,161 - INFO: Output format set to csv
2023-08-04 09:37:07,271 - INFO: reading line {"approved_by":null,"archived":false,"author":"buzzlit","author_flair_background_color":null,"author_flair_css_class":null,"author_flair_richtext":[],"author_flair_template_id":null,"author_flair_text":null,"author_flair_text_color":null,"author_flair_type":"text","banned_by":null,"can_gild":true,"category":null,"clicked":false,"content_categories":null,"contest_mode":false,"created":1527299589.0,"created_utc":1527270789,"distinguished":null,"domain":"self.Mechwarrior5","downs":0,"edited":false,"gilded":0,"hidden":false,"hide_score":false,"id":"8m3qlb","is_crosspostable":false,"is_original_content":false,"is_reddit_media_domain":false,"is_self":true,"is_video":false,"likes":null,"link_flair_css_class":null,"link_flair_richtext":[],"link_flair_text":null,"link_flair_text_color":"dark","link_flair_type":"text","locked":false,"media":null,"media_embed":{},"media_only":false,"mod_reports":